In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd, numpy as np

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [3]:
X_train = torch.tensor(pd.read_csv("./data/x_train.csv").to_numpy(), dtype=float)
y_train = torch.tensor(pd.read_csv("./data/y_train.csv").to_numpy(), dtype=float)

X_test = torch.tensor(pd.read_csv("./data/x_test.csv").to_numpy(), dtype=float)
y_test = torch.tensor(pd.read_csv("./data/y_test.csv").to_numpy(), dtype=float)

In [4]:
# -----------------------------
# INITIALIZE MODEL, LOSS, AND OPTIMIZER
# -----------------------------

# Create an instance of your model (a simple linear regression model here)
# This model will learn a relationship like: y = w*x + b
model = nn.Linear(in_features=X_train.shape[1], out_features=1, dtype=float)

# Define the loss function (how we measure the model’s error)
# nn.MSELoss() = Mean Squared Error → average of squared differences between predicted and actual values
criterion = nn.MSELoss()

# Define the optimizer — this updates model weights to reduce the loss
# SGD = Stochastic Gradient Descent, a common optimization algorithm
# model.parameters() = tells optimizer which parameters to update (weights & biases)
# lr=0.01 = learning rate, controls how big a step we take each update
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [5]:
# -----------------------------
# TRAINING LOOP FOR A MODEL
# -----------------------------

# Set a small tolerance value — if the loss stops changing by more than this, we’ll stop training early
tol = 1e-5

# Set the maximum number of epochs (full passes through the dataset)
# 1e7 = 10 million (a very large number, used here as an upper limit)
epochs = int(1e7)

# Store the previous loss value; start with infinity so any real loss will be smaller
prev_loss = float("inf")

# Loop through each epoch (training iteration)
for epoch in range(epochs):

    # ---- Forward pass ----
    # Feed the training data (X_train) to the model to get predictions (outputs)
    outputs = model(X_train)

    # Calculate how far off the predictions are from the true labels (y_train)
    # The 'criterion' defines the loss function (e.g., MSE, cross-entropy, etc.)
    loss = criterion(outputs, y_train)

    # ---- Backward pass & optimization ----
    # Clear (reset) any gradients from the previous step
    optimizer.zero_grad()

    # Compute new gradients for each model parameter based on the current loss
    loss.backward()

    # Update model parameters (weights and biases) using the optimizer
    optimizer.step()

    # ---- Check progress ----
    # Convert the loss (a PyTorch tensor) to a plain Python number
    curr_loss = loss.item()

    # Print the current epoch number and loss every 50 epochs to track progress
    if (epoch + 1) % 50 == 0:
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {curr_loss:.4f}")

    # ---- Early stopping condition ----
    # If the loss hasn’t changed much compared to the last epoch, stop training
    if abs(curr_loss - prev_loss) <= tol:
        break

    # Save current loss as previous loss for the next iteration
    prev_loss = curr_loss

Epoch [50/10000000], Loss: 381.0717
Epoch [100/10000000], Loss: 98.1902
Epoch [150/10000000], Loss: 54.8435
Epoch [200/10000000], Loss: 37.4709
Epoch [250/10000000], Loss: 26.7108
Epoch [300/10000000], Loss: 19.4895
Epoch [350/10000000], Loss: 14.5849
Epoch [400/10000000], Loss: 11.2477
Epoch [450/10000000], Loss: 8.9762
Epoch [500/10000000], Loss: 7.4300
Epoch [550/10000000], Loss: 6.3775
Epoch [600/10000000], Loss: 5.6611
Epoch [650/10000000], Loss: 5.1734
Epoch [700/10000000], Loss: 4.8414
Epoch [750/10000000], Loss: 4.6155
Epoch [800/10000000], Loss: 4.4617
Epoch [850/10000000], Loss: 4.3570
Epoch [900/10000000], Loss: 4.2857
Epoch [950/10000000], Loss: 4.2372
Epoch [1000/10000000], Loss: 4.2041
Epoch [1050/10000000], Loss: 4.1817
Epoch [1100/10000000], Loss: 4.1664
Epoch [1150/10000000], Loss: 4.1560
Epoch [1200/10000000], Loss: 4.1489
Epoch [1250/10000000], Loss: 4.1440
Epoch [1300/10000000], Loss: 4.1408
Epoch [1350/10000000], Loss: 4.1385
Epoch [1400/10000000], Loss: 4.1370
Epo

In [6]:
torch.save(model, "./model/model.pt")

In [7]:
model = torch.load("./model/model.pt", weights_only=False)

In [8]:
y_pred = model(X_test)

y_test_flat, y_pred_flat = y_test.numpy().flatten(), y_pred.detach().numpy().flatten()

mse = mean_squared_error(y_test_flat, y_pred_flat)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test_flat, y_pred_flat)
r2 = r2_score(y_test_flat, y_pred_flat)

print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"R2 Score: {r2:.4f}")

Mean Squared Error (MSE): 4.2328
Root Mean Squared Error (RMSE): 2.0574
Mean Absolute Error (MAE): 1.6441
R2 Score: 0.9889
